In [1]:
#Libraries
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.parser import parse
from functools import reduce
from random import choice
from requests import get
from scipy.stats import iqr
from sklearn import preprocessing
from sqlalchemy import create_engine
from zipfile import ZipFile
import gzip
import itertools
import locale
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import re
import requests
import seaborn as sns
#import wget
sns.set()

C:\Users\ellio\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
url = 'https://www.bhhsfloridarealty.com/AgentSearch/Results.aspx?SearchType=agent&FirstName=&LastName=&OfficeName=&Address=&City=&State=&Country=-32768&Zip=&Languages=&Titles=&Specialties=&Accreditations=&Areas=&rpp=50&page=2&SortOrder='
response = get(url)

In [3]:
html_soup = BeautifulSoup(response.text, 'html.parser')
html_soup


<!DOCTYPE html>

<html lang="en">
<head id="Master_Head1"><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><!-- Build Number: 2020.7.13.353-hx | Server: SWBRW-OLA20 | Design: /RECoTemplates/Freedom03 -->
<meta content="Real Estate Agent Search Results | BHHS Florida Realty" name="title"/>
<meta content="real estate listings,real estate,realty,homes for sale,houses for sale,mortgages,home loans,finance,financing,properties,single family homes,virtual tour,new construction,new homes,first time homebuyers,home buying checklist,home selling checklist,for sale by owner,house hunting,home improvement,waterfront property,lakefront property,lakeshore property,real estate agents,real estate broker,realtors,mls listings,residential,home building,reality,swimming pools,seminars,luxury homes,exceptional,distinctive,distinguished,condominiums,condos,town houses,townhouses,twin homes,twinhomes,duplexes,

In [12]:

agent_containers = html_soup.find_all('div', class_="ao-info-container")
agent_containers[0]
first = agent_containers[0]

In [32]:
len(agent_containers)

50

In [33]:
type(agent_containers)

bs4.element.ResultSet

In [31]:
first

<div class="ao-info-container"><div class="ao-info-r1 rui-inner-row">
<div class="ao-info-c1">
<h3><a class="ao_results_icon_text A detail-page" href="/AgentSearch/LoriAurora-22105149">Lori Aurora</a></h3>
<div class="ao-acc"></div>
<div class="ao-address">
<span class="ao-address">8319 Market Street</span>
                    
                    Lakewood Ranch, Florida 34202
                </div>
<div class="ao-phone" id="ao-phone">603-661-6116</div>
<div class="ao-phone" id="ao-cell">603-661-6116</div>
</div>
<div class="ao-info-c2">
<div class="ao-office"><a class="ao-office" href="https://www.bhhsfloridarealty.com/Office/SarasotaLakewoodRanch-526060" target="_blank">Sarasota / Lakewood Ranch</a>
</div>
</div>
<div class="ao-info-c3"><div class="ao-social">
</div>
<div class="ao-realping"></div>
</div>
</div>
<div class="ao-info-r2 rui-inner-row">
<div class="ao_languages_container" style="white-space: nowrap;">Languages: English</div>
</div>
<div class="ao-info-r3 rui-inner-row">

In [21]:
first.find_all('div')

[<div class="ao-info-r1 rui-inner-row">
 <div class="ao-info-c1">
 <h3><a class="ao_results_icon_text A detail-page" href="/AgentSearch/LoriAurora-22105149">Lori Aurora</a></h3>
 <div class="ao-acc"></div>
 <div class="ao-address">
 <span class="ao-address">8319 Market Street</span>
                     
                     Lakewood Ranch, Florida 34202
                 </div>
 <div class="ao-phone" id="ao-phone">603-661-6116</div>
 <div class="ao-phone" id="ao-cell">603-661-6116</div>
 </div>
 <div class="ao-info-c2">
 <div class="ao-office"><a class="ao-office" href="https://www.bhhsfloridarealty.com/Office/SarasotaLakewoodRanch-526060" target="_blank">Sarasota / Lakewood Ranch</a>
 </div>
 </div>
 <div class="ao-info-c3"><div class="ao-social">
 </div>
 <div class="ao-realping"></div>
 </div>
 </div>, <div class="ao-info-c1">
 <h3><a class="ao_results_icon_text A detail-page" href="/AgentSearch/LoriAurora-22105149">Lori Aurora</a></h3>
 <div class="ao-acc"></div>
 <div class="ao-ad

In [ ]:
([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)

1.   First Name
2.  Last name
3.  Phone number
4.  Email address
5.  City

In [35]:

# NAME

#<h3><a href="/AgentSearch/LoriAurora-22105149" class="ao_results_icon_text A detail-page">Lori Aurora</a></h3>
html_soup.find_all('a', class_="ao_results_icon_text A detail-page")
# regex first part
# .*AgentSearch\W[a-zA-Z0-9._-]+\W{2}
# or   ^.*AgentSearch\W[a-zA-Z0-9._-]+\W{2}                starts with ^

# regex end bit
#  \W\Wa\W$



[<a class="ao_results_icon_text A detail-page" href="/AgentSearch/LoriAurora-22105149">Lori Aurora</a>,
 <a class="ao_results_icon_text A detail-page" href="/AgentSearch/FrancesAyala-22563239">Frances Ayala</a>,
 <a class="ao_results_icon_text A detail-page" href="/AgentSearch/AlfredoAyme-22134737">Alfredo Ayme</a>,
 <a class="ao_results_icon_text A detail-page" href="/AgentSearch/MarleenAyton-15475197">Marleen Ayton</a>,
 <a class="ao_results_icon_text A detail-page" href="/AgentSearch/OmarAzam-22570464">Omar Azam</a>,
 <a class="ao_results_icon_text A detail-page" href="/AgentSearch/TonyAzzarello-22197209">Tony Azzarello</a>,
 <a class="ao_results_icon_text A detail-page" href="/AgentSearch/FrankBabusik-15476590">Frank Babusik</a>,
 <a class="ao_results_icon_text A detail-page" href="/AgentSearch/VictoriaBacePerdomoPA-22549170">Victoria Bace Perdomo, P.A.</a>,
 <a class="ao_results_icon_text A detail-page" href="/AgentSearch/HollyBaconLLC-19473485">Holly Bacon, LLC</a>,
 <a class="ao

In [ ]:
# phone
# <div class="ao-phone" id="ao-phone"></div>
# <div class="ao-phone" id="ao-cell"></div>

In [42]:
html_soup.find_all('div', class_="ao-phone")

[<div class="ao-phone" id="ao-phone">603-661-6116</div>,
 <div class="ao-phone" id="ao-cell">603-661-6116</div>,
 <div class="ao-phone" id="ao-phone"></div>,
 <div class="ao-phone" id="ao-cell"></div>,
 <div class="ao-phone" id="ao-phone">305-586-5656</div>,
 <div class="ao-phone" id="ao-cell">305-586-5656</div>,
 <div class="ao-phone" id="ao-phone"></div>,
 <div class="ao-phone" id="ao-cell"></div>,
 <div class="ao-phone" id="ao-phone"></div>,
 <div class="ao-phone" id="ao-cell"></div>,
 <div class="ao-phone" id="ao-phone">419-303-1692</div>,
 <div class="ao-phone" id="ao-cell">419-303-1692</div>,
 <div class="ao-phone" id="ao-phone">(772) 349-2284</div>,
 <div class="ao-phone" id="ao-cell">(772) 349-2284</div>,
 <div class="ao-phone" id="ao-phone"></div>,
 <div class="ao-phone" id="ao-cell"></div>,
 <div class="ao-phone" id="ao-phone">(239) 404-8994</div>,
 <div class="ao-phone" id="ao-cell">(239) 404-8994</div>,
 <div class="ao-phone" id="ao-phone"></div>,
 <div class="ao-phone" id=

In [34]:
# email
re.search(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', str(first.find_all('div')))

<re.Match object; span=(2576, 2608), match='loriaurora@bhhsfloridarealty.net'>

In [ ]:
# for value in dictionary (of which there are 32):
    # for 